In [22]:
#importation des bibliothèques

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from math import sqrt
from sklearn.model_selection import KFold, cross_validate
from sklearn.ensemble import RandomForestRegressor

In [15]:
#lecture des tableaux

df = pd.read_csv('tableau_pour_apprentissage.csv', sep=';')
df2 = pd.read_csv('tableau_a_remplir.csv', sep=';')

#on garde que les lignes du tableau pour apprentissage contenant les CHE
df_masque = df[df['cespece'] == 'CHE']

#on fait en sorte d'avoir autant de lignes dans le tableau pour apprentissage que dans le tableau à remplir
df_sample = df_masque.sample(n = 468)

#on garde que les données quantitatives du tableau d'apprentissage
df_quanti = df_sample[['Secteur','NumStation','Month','Year']]

In [16]:
#centre-réduction des données

scaler = StandardScaler() 
scaler.fit(df_quanti) 
df_scaled = pd.DataFrame(scaler.transform(df_quanti), columns=['Secteur','NumStation','Month','Year'])

In [17]:
#adaptation des données

df_scaled_nomiss = df_scaled.dropna()
df_features = df_scaled_nomiss.iloc[:,:-1]
df_label = df_sample['taille (cm)']

In [18]:
#séparation des données

i = 42
X_train, X_test, y_train, y_test = train_test_split(df_features, df_label, test_size=0.8, random_state=i)

In [19]:
#entraînement du modèle
model = RandomForestRegressor()
model.fit(X_train, y_train)

#prédictions
pred_train_y = model.predict(X_train)
pred_test_y = model.predict(X_test)

In [20]:
#validation croisée via CV

max_depth_val = 10
max_features_val = 'sqrt'
min_samples_leaf_val = 3
min_samples_split_val = 5
n_estimators_val = 200
cv = KFold(n_splits=5, random_state=1, shuffle=True)
rf = RandomForestRegressor(n_estimators=n_estimators_val,
                           max_depth=max_depth_val,
                           min_samples_split=min_samples_split_val,
                           min_samples_leaf=min_samples_leaf_val,
                           n_jobs=-1,
                           random_state=1)
scores = cross_validate(rf, df_features, df_label, scoring=['r2', 'neg_mean_squared_error'],  cv=cv, n_jobs=-1)

In [21]:
#remplissage du dataframe

df2['taille (cm)'] = model.predict(X_test)
df2.to_csv("fichier_finalCV.csv", index=False)